In [ ]:
## alomst finished code, takes presaved datsegs and appends to it such that 
# all sequences have its mirror counterpart.

# NOTE: datflat is not updated, could add that to the code, although may not be imoprtant if don't use datflat.

## given parse object (.pickel) get all reversed parses too

%load_ext autoreload
%autoreload 2

from analysis.utils import *
from analysis.parse import *
from analysis.analy import *

EXPT = "S12.10"
DAT = loadCheckpoint(EXPT, userealnames=True, loadparse=True, suppressPrint=True, loadbehavior=False)

# load datflat and datseg

stim = "S12_139"
datflat = DATloadDatFlat(DAT, stim)
datseg = DATloadDatSeg(DAT, stim)

assert len(datflat)==len(datseg), "assuming they are perfectly matched"
sequencelist = [[d["codes_unique"] for d in dseg] for dseg in datseg]

# convert to strings
# sequencelist_reverse = ["_".join(s[::-1]) for s in sequencelist]
# sequencelist = ["_".join(s) for s in sequencelist]

def getFwRevSeqStrings(seq):
    """seq is list of strings. will output one long strong for fw and rev.
    confirmed that they ware mirror image (i.e. reverses tokens too)"""
    
    # get fw
    seqfw = "_".join(seq)
    
    # get rev
    tmp = seq
#     tmp = [s[::-1] for s in seq] # flip each token
    tmp = tmp[::-1]
    seqrev = "_".join(tmp)

    return seqfw, seqrev

    
sequencelist = [[d["codes_unique"] for d in dseg] for dseg in datseg]

sequences_fw = []
sequences_rev = []
for seq in sequencelist:
    
    f, r = getFwRevSeqStrings(seq)
    sequences_fw.append(f)
    sequences_rev.append(r)

# get all unique sequences
sequences_fw.extend(sequences_rev)
sequences_all = set(sequences_fw)
sequences_to_add = []
for seq in sequences_all:
    
    # -- check thru all datsgs for this sequence
    found=False
    for df, ds in zip(datflat, datseg):
        
        # get sequence
        fw = getFwRevSeqStrings([d["codes_unique"] for d in ds])[0]
        if seq==fw:
            found=True
            break
    if not found:
        # then add this sequence:
        sequences_to_add.append(seq)


# -- get dictionayr that can use to recompose new sequence
seqdict = {}
for ds in datseg[0]:
    seqdict[ds["codes_unique"]]=ds
        

for seq in sequences_to_add:
    # get list of tokens
    seqlist = seq.split("_")
    
    dsegthis = [seqdict[scode] for scode in seqlist]
    datseg.append(dsegthis)

# TO DO:
# SAVE THE DATSEG BACK.
